In [8]:
from typing import Tuple, List, Dict, Optional, Set
import heapq
import numpy as np

### Task 1

In [9]:

maze: List[List[int]] = [
  [0, 0, 1, 0, 0],
  [0, 0, 0, 0, 0],
  [0, 0, 1, 0, 1],
  [0, 0, 1, 0, 0],
  [0, 0, 0, 1, 0]
]

def heuristic(source: Tuple[int, int], goal: Tuple[int, int]) -> int:
  return abs(source[0] - goal[0]) + abs(source[1] - goal[1])

goals: Set[Tuple[int, int]] = set([(4, 4), (2, 3), (0, 4)])

type DistanceDict = Dict[Tuple[int, int], Tuple[int, Optional[Tuple[int, int]]]]
def best_first_search(maze: List[List[int]], source: Tuple[int, int], goal: Tuple[int, int]) -> List[Tuple[int, int]]:
  dists: DistanceDict = {source: (0, None)}
  pq = [source]
  n, m = len(maze), len(maze[-1])

  def construct_path() -> List[Tuple[int, int]]:
    path = [goal]
    curr: Optional[Tuple[int, int]] = goal
    while curr:
      parent = dists[curr][1]
      path.append(parent)
      curr = parent
    return path[::-1][1:]

  while pq:
    r, c = heapq.heappop(pq)

    for dr, dc in [(0,1), (1,0), (0,-1), (-1,0)]:
      nr, nc = r + dr, c + dc
      if not (0 <= nr < n and 0 <= nc < m) or maze[nr][nc] == 1:
        continue

      f = heuristic((nr, nc), goal)
      if f < dists.get((nr, nc), (float('inf'), None))[0]:
        dists[(nr, nc)] = (f, (r, c))
        heapq.heappush(pq, (nr, nc))

  return construct_path()

def display_path(maze: List[List[int]], path: List[Tuple[int, int]]):
  for r, c in path:
    maze[r][c] = 2
  print(*map(lambda row: ' '.join(map(str, row)), maze), sep="\n", end="\n\n")

for goal in goals:
  path = best_first_search(maze, (0, 0), goal)
  display_path([row[:] for row in maze], path)

2 2 1 0 0
0 2 2 2 0
0 0 1 2 1
0 0 1 2 2
0 0 0 1 2

2 2 1 0 0
0 2 2 2 0
0 0 1 2 1
0 0 1 0 0
0 0 0 1 0

2 2 1 2 2
0 2 2 2 0
0 0 1 0 1
0 0 1 0 0
0 0 0 1 0



### Task 2

In [20]:
import heapq
import random
import time
from typing import Dict, List, Tuple, Optional

Node = Tuple[int, int]
Edge = Tuple[Node, int]
Graph = Dict[Node, List[Edge]]
Key = Tuple[float, float]

class IncrementalAStar:
  def __init__(self):
    self.g_scores: Dict[Node, float] = {}
    self.rhs: Dict[Node, float] = {}
    self.open_list: List[Tuple[Key, Node]] = []
    self.graph: Graph = {}
    self.start: Optional[Node] = None
    self.goal: Optional[Node] = None

  def initialize(self, graph: Graph, start: Node, goal: Node):
    self.graph = graph
    self.start = start
    self.goal = goal
    self.g_scores = {node: float('inf') for node in graph}
    self.rhs = {node: float('inf') for node in graph}
    self.rhs[goal] = 0
    self.open_list = []
    heapq.heappush(self.open_list, (self.calculate_key(goal), goal))

  def calculate_key(self, node: Node) -> Key:
    g_rhs = min(self.g_scores[node], self.rhs[node])
    return (g_rhs + self.heuristic(node, self.start), g_rhs)

  def heuristic(self, node: Node, target: Node) -> int:
    return abs(node[0] - target[0]) + abs(node[1] - target[1])

  def update_vertex(self, node: Node):
    if self.g_scores[node] != self.rhs[node]:
      heapq.heappush(self.open_list, (self.calculate_key(node), node))
    else:
      self.open_list = [entry for entry in self.open_list if entry[1] != node]
      heapq.heapify(self.open_list)

  def compute_shortest_path(self):
    while self.open_list and (self.open_list[0][0] < self.calculate_key(self.start) or self.rhs[self.start] != self.g_scores[self.start]):
      _, current = heapq.heappop(self.open_list)

      self.g_scores[current] = self.rhs[current] if self.g_scores[current] > self.rhs[current] else float('inf')
      for neighbor, cost in self.graph.get(current, []):
        self.rhs[neighbor] = min(self.rhs[neighbor], self.g_scores[current] + cost)
        self.update_vertex(neighbor)
      self.update_vertex(current)

  def find_path(self) -> Optional[List[Node]]:
    path = []
    current = self.start
    while current != self.goal:
      path.append(current)
      if current not in self.graph or self.g_scores[current] == float('inf'):
        return None
      current = min(self.graph[current], key=lambda x: self.g_scores[x[0]])[0]
    path.append(self.goal)
    return path

  def modify_graph(self):
    for node in random.sample(list(self.graph.keys()), k=random.randint(1, len(self.graph))):
      if not self.graph[node]:
        continue

      neighbor, _ = random.choice(self.graph[node])
      new_cost = random.randint(1, 10)
      print(f"Updating edge {node} -> {neighbor} cost to {new_cost}")
      self.graph[node] = [
        (n, new_cost if n == neighbor else c)
        for n, c in self.graph[node]
      ]
      self.update_vertex(node)

graph = {
  (0, 0): [((1, 0), 1), ((0, 1), 1)],
  (1, 0): [((0, 0), 1), ((1, 1), 1)],
  (0, 1): [((0, 0), 1), ((1, 1), 1)],
  (1, 1): [((1, 0), 1), ((0, 1), 1)],
}

start = (0, 0)
goal = (1, 1)
algo = IncrementalAStar()
algo.initialize(graph, start, goal)

for i in range(2):
  print(f"\nIteration {i+1}:")
  algo.compute_shortest_path()
  path = algo.find_path()
  print("Shortest Path:", path)
  time.sleep(1)
  algo.modify_graph()



Iteration 1:
Shortest Path: [(0, 0), (1, 0), (1, 1)]
Updating edge (0, 0) -> (0, 1) cost to 1
Updating edge (0, 1) -> (0, 0) cost to 8
Updating edge (1, 1) -> (0, 1) cost to 3
Updating edge (1, 0) -> (1, 1) cost to 4

Iteration 2:
Shortest Path: [(0, 0), (1, 0), (1, 1)]
Updating edge (1, 1) -> (0, 1) cost to 2
Updating edge (0, 0) -> (1, 0) cost to 7


### Task 3

In [19]:
from typing import List, Tuple
import heapq

grid = [
  ["S", ".", ".", "D1", "#"],
  [".", "#", ".", ".", "."],
  [".", ".", ".", "#", "D2"],
  ["D3", ".", "#", ".", "."],
  [".", "D4", ".", ".", "."],
]

nodes = {
  "D1": (0, 3, 5, 15),
  "D2": (2, 4, 8, 12),
  "D3": (3, 0, 10, 20),
  "D4": (4, 1, 7, 14),
}

start = (0, 0)

def distance(p1: Tuple[int, int], p2: Tuple[int, int]) -> int:
  return abs(p1[0] - p2[0]) + abs(p1[1] - p2[1])

def greedy_best_first_search(grid: List[List[str]], start: Tuple[int, int], nodes: dict):
  rows, cols = len(grid), len(grid[0])
  visited = set()
  pq = []
  route = []
  curr_time = 0
  heapq.heappush(pq, (0, start, curr_time))

  while pq:
    _, current, curr_time = heapq.heappop(pq)
    if current in visited:
      continue
    visited.add(current)
    route.append(current)

    for name, (x, y, t_start, t_end) in list(nodes.items()):
      if (x, y) == current and t_start <= curr_time <= t_end:
        nodes.pop(name)
        break

    for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
      nx, ny = current[0] + dx, current[1] + dy
      if not (0 <= nx < rows and 0 <= ny < cols) or (nx, ny) in visited or grid[nx][ny] == "#":
        continue

      heuristic = float("inf")
      for name, (tx, ty, t_start, t_end) in nodes.items():
        travel_time = curr_time + 1 + distance((nx, ny), (tx, ty))
        if travel_time <= t_end:
          time_priority = 1 / (t_end - travel_time + 1)
          dist_priority = distance((nx, ny), (tx, ty))
          heuristic = min(heuristic, 0.7 * time_priority + 0.3 * dist_priority)
      heapq.heappush(pq, (heuristic, (nx, ny), curr_time + 1))

  return route

greedy_best_first_search(grid, start, nodes)

[(0, 0),
 (1, 0),
 (2, 0),
 (3, 0),
 (3, 1),
 (4, 1),
 (4, 0),
 (4, 2),
 (2, 1),
 (0, 1),
 (0, 2),
 (0, 3),
 (1, 3),
 (1, 4),
 (2, 4),
 (3, 4),
 (1, 2),
 (2, 2),
 (4, 3),
 (3, 3),
 (4, 4)]